In [3]:
import numpy as np
import matplotlib.pyplot as plt
from nn import io, nn, preprocess
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits

TypeError: Cannot instantiate typing.Union